# Cepstrum Plotting

A continuation of PSD_by_Type notebook. This notebook was started when I was tasked to make spectrograms using the cepstrum of a signal. This should not be done with the OOIPY spectrogram plotting functions but should instead just be hardcoded using the calculated spectrogram functions.

## Imports & Functions

In [1]:
# data
import ooipy
from ooipy.tools import ooiplotlib as ooiplt
import pandas as pd
import numpy as np
#import functions as fn
# plotting
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from matplotlib.backends.backend_agg import FigureCanvasAgg
# IO
from io import BytesIO
import io
import json
from scipy.io import wavfile
import re

from scipy.signal import welch

def cepstrum_maker(S, t_sections=20):
    w = int(len(S)/t_sections) 
    time_windows = []
    for i in range(1, t_sections+1):
        """if (i == 10):
            window = S[((i-1)*w):]
        else:
            window = S[((i-1)*w):(i*w)]"""
        # we lose out on very last value
        window = S[((i-1)*w):(i*w)]
        time_windows.append(window)

    cf_windows = []
    for idx, wind in enumerate(time_windows):
        #print(f'window {idx}: {wind}')
        #print(f'Window {idx} shape is {wind.shape}')
        # compute cf
        cf = (np.log(np.abs(np.fft.fft(wind))))
        icf = np.real(np.fft.ifft((cf)))
        cf_windows.append(icf[:int(len(cf)/2)]) # only need half, nyquist
        #cf_windows.append(cf)
    return cf_windows
    


def get_cepsPSD_dict(meta_data_CSV_path, location, print_options=False):
    """
    From given csv metadata path and location string, give a dictionary of PSD values using the cepstrum
    """
    rawdf = pd.read_csv('data/'+location+'/'+meta_data_CSV_path, sep=',')
    if (print_options):
        options = ['Cargo', 'Tanker', 'Fishing type']
        df = rawdf[rawdf['ud_group'].isin(options)]
        df = df.reset_index(drop=True)
        #df = rawdf[(df['ud_group']=='Cargo') | (df['ud_group']=='Tanker') | (df['ud_group']=='Fishing type')]
        # EC is the only one with missing ship type and only 4 of them, just ignore those
        # .dropna() followed by .groupby() cleans and groups
        grouped_df = df.dropna(subset=['ud_group', 'VESSEL TYPE']).groupby(['ud_group', 'VESSEL TYPE'])
        #grouped_df.sum().tail(20)
        mean_dict = {}  

        # Create average data
        for group_name, df_group in grouped_df:
            print(group_name)

    options = ['Cargo', 'Tanker', 'Fishing type']
    df = rawdf[rawdf['VESSEL TYPE'].isin(options)]
    df = rawdf.reset_index(drop=True)
    grouped_df = df.dropna(subset=['ud_group', 'VESSEL TYPE']).groupby(['ud_group', 'VESSEL TYPE'])

    # https://stackoverflow.com/questions/27405483/how-to-loop-over-grouped-pandas-dataframe
    PSDs = {}
    for group_name, df_group in grouped_df:
        PSD_freq = []
        PSD_val = []
        dictPSD = { 'frequencies': PSD_freq, 'values': PSD_val}

        # for-loop to compile list of PSD data objects pulled from column
        for row_index, row in df_group.iterrows():
            inst_id = df['instance_id'].iloc[row_index]
            # get pickle files (DL locally? Call some API?)
            # local storage method
            data_path = 'data/' + location + '/'
            wavfilepath = data_path + 'Audio/' + inst_id + '.wav'
            # try-except deals with missing pickles
            try:
                output = wavfile.read(wavfilepath)
                S = output[1]
                total_cf = np.log(np.abs(np.fft.fft(S)))
                total_quefrec = np.fft.ifft(total_cf)
                cf = cepstrum_maker(S)
                icf = np.fft.ifft(cf)
                #print(cf)
                obj = ooipy.hydrophone.basic.HydrophoneData(data=np.real(icf), header=None, node="Axial_Base")
                obj.stats.sampling_rate = 200
                obj.stats.channel = 'HDH'
                obj.compute_psd_welch(L = 256, overlap=0.3)
                #print(obj.stats)
                psd = obj.psd   
                dictPSD['frequencies'].append(psd.freq)
                dictPSD['values'].append(psd.values)
            except FileNotFoundError:
                pass
        PSDs.update({group_name: dictPSD})
    return PSDs

def sine_maker(frequency=440.0, duration=1.0):
        """
        This will generate a sine wave with minimum of 1 second duration.
        """

        BITRATE = 200.
        grain = round(BITRATE / frequency)
        points = grain * round(BITRATE * duration / grain) + 1
        duration = points / BITRATE

        data = np.zeros(int(BITRATE * max(duration, 1.0)))

        #try:
        times = np.linspace(0, duration, points, endpoint=False)
        data[:points] = np.sin(times * frequency * 2 * np.pi)
        data = np.array((data + 1.0) * 127.5, dtype=np.int8)
        """except:  # do it without numpy
            data = ''
            omega = 2.0*np.pi*frequency/BITRATE
            for i in range(points):
                data += chr(int(127.5*(1.0+np.sin(float(i)*omega))))"""
        #self.stream.write(data)
        return data

def signal_maker(freq_list, duration):
    signal = 0
    for freq in freq_list:
        signal = signal + sine_maker(frequency=freq, duration=duration)
    return signal

def spectrogram_maker(signal_arr):
    obj = ooipy.hydrophone.basic.HydrophoneData(data=signal_arr, header=None, node="Axial_Base")
    obj.stats.sampling_rate = 200
    obj.stats.channel = 'HDH'
    spectrogram = obj.compute_spectrogram(L = 256,avg_time=10, overlap=0.9)
    fmin=0
    fmax=100
    ooiplt.plot_spectrogram(spectrogram,plot=True,save=False,fmin=fmin,fmax=fmax,vmax=110)#,filename=path+inst_id+'.png')
    

## Spectrogram

### For-Loop to make spectrogram of all instances

In [ ]:
rawdf = pd.read_csv('data/'+'Axial_Base'+'/'+'AB_isolated_ais_10m_5_20.csv', sep=',')
options = ['Cargo', 'Tanker', 'Fishing type']
df = rawdf[rawdf['VESSEL TYPE'].isin(options)]
df = rawdf.reset_index(drop=True)
grouped_df = df.dropna(subset=['ud_group', 'VESSEL TYPE']).groupby(['ud_group', 'VESSEL TYPE'])

for row_index, row in df.iterrows():
    inst_id = df['instance_id'].iloc[row_index]

    wavfilepath = 'data/Axial_Base/' + 'Audio/' + inst_id + '.wav'
    try:
        output = wavfile.read(wavfilepath)
        S = output[1]
        t_sections = 20
        cf_windows = cepstrum_maker(S, t_sections=t_sections)

        Fs = 200
        #t = np.arange(t_sections * 60 * Fs /t_sections/2/2) 
        min_per_section = 10/t_sections
        t = np.arange((min_per_section*60*Fs)/2) #/2 for cutting in half for nyquist

        fig, ax = plt.subplots(figsize=(14, 9))
        CS = ax.contourf(np.arange(0,10, 0.5), t/Fs, np.asarray((cf_windows)).T, cmap=plt.cm.plasma)
        plt.ylim((-.01))
        ax.set_title('Cepstrogram '+ inst_id)
        fig.colorbar(CS)
        ax.set_ylabel('quefrency (s)')
        ax.set_xlabel('Time (min)')

        filename = "data/"+'Axial_Base' + '/Cepstrum/Spectrograms/'+inst_id+'.png'
        plt.savefig(filename)
        plt.close(fig)
        #plt.show()
        
    except (FileNotFoundError, TypeError) as e:
        pass


### For-Loop to make spectrogram of ONE instances

In [14]:
rawdf = pd.read_csv('data/'+'Axial_Base'+'/'+'AB_isolated_ais_10m_5_20.csv', sep=',')
options = ['Cargo', 'Tanker', 'Fishing type']
df = rawdf[rawdf['VESSEL TYPE'].isin(options)]
df = rawdf.reset_index(drop=True)
grouped_df = df.dropna(subset=['ud_group', 'VESSEL TYPE']).groupby(['ud_group', 'VESSEL TYPE'])

inst_id = df['instance_id'].iloc[1000]

wavfilepath = 'data/Axial_Base/' + 'Audio/' + inst_id + '.wav'
try:
    output = wavfile.read(wavfilepath)
    S = output[1]
    t_sections = 20
    cf_windows = cepstrum_maker(S, t_sections=t_sections)

    Fs = 200
    #t = np.arange(t_sections * 60 * Fs /t_sections/2/2) 
    min_per_section = 10/t_sections
    t = np.arange((min_per_section*60*Fs)/2) #/2 for cutting in half for nyquist

    #fig, ax = plt.subplots(figsize=(14, 9))
    plt.contourf(np.arange(0,10, 0.5), t/Fs, np.asarray((cf_windows)).T, cmap=plt.cm.plasma, vmin=-1.5, vmax=3)
    plt.ylim((-.01))
    plt.title('Cepstrogram '+ inst_id)
    plt.colorbar()
    plt.ylabel('quefrency (s)')
    plt.xlabel('Time (min)')

    filename = "data/"+'Axial_Base' + '/Cepstrum/Spectrograms/'+inst_id+'.png'
    #plt.savefig(filename)
    #plt.close(fig)
    plt.show()

except (FileNotFoundError, TypeError) as e:
    pass

### Legacy code; observed icf and cf values for one time window (last window)

In [ ]:
#plt.plot(S[:int(len(S)/10)], label='original signal')
plt.plot(icf, label='icf, time domain')
plt.plot(cf, label='cf, freq domain')

plt.legend()
plt.show()

## Spectrogram Recreation

From Spectrogram data/plots, we will attempt to recreate it using added sine waves or truncated cepstrums

### Adding Sine Waves from graph (values taken visually)

In [10]:
dur = 60*10

# weighted sum of waves
testa = sine_maker(frequency=5.0, duration=dur)
testb = sine_maker(frequency=45.0, duration=dur)
testc = 0.5*sine_maker(frequency=52.0, duration=dur)
testd = sine_maker(frequency=60.0, duration=dur)
teste = 0.6*sine_maker(frequency=68.0, duration=dur)

# unweighted sum of waves
signal = signal_maker([5, 45, 52, 60,68], dur)
signal1 = testa+testb+testc+testd+teste

plt.plot(signal1)
plt.show()

In [11]:
spectrogram_maker(signal1)
plt.show()

c:\Users\UserALai\miniconda3\lib\site-packages\ooipy\tools\ooiplotlib.py:218: UserWarning: The following kwargs were not used by contour: 'plot', 'save', 'fmin', 'fmax', 'filename', 'title', 'xlabel', 'xlabel_rot', 'xlabel_format', 'ylabel', 'vdelta', 'vdelta_cbar', 'figsize', 'res_reduction_time', 'res_reduction_freq', 'dpi', 'fontsize', 'extend_type', 'logy'
  ax.contourf(


In [67]:
signal.shape

(120001,)

### Zeroing out parts of Cepstrum

In [15]:
# run one instance spectrogram code to get cepstrogram (stored as cf_windows)

arr = np.asarray((cf_windows))

# truncate and use lower quefrency
arr[:, int(len(t)/3):] = 0
cf_windows_rec = (arr.tolist())
time_windows_rec = []

# undo cepstrum process
for idx, wind in enumerate(cf_windows_rec):
    # did not account for np.abs() and np.real()
    cf_transformed = np.exp(np.fft.fft(wind)) 

    # undo truncation from nyquist
    # by adding on reflected frequency to end of frequency
    time_wind = np.fft.ifft(np.append(cf_transformed, np.flip(cf_transformed)))
    time_windows_rec.append(time_wind)

# put windows together 
recreated_signal = np.concatenate(time_windows_rec)
# and add on extra value(?)
recreated_signal2 = np.append(recreated_signal, np.mean(recreated_signal))
#eliminate extremely sharp peaks
recreated_signal2 = recreated_signal2 * (np.abs(recreated_signal2)<500)
spectrogram_maker(np.real(recreated_signal2))
plt.show()

In [5]:
plt.plot(arr)
plt.show()

In [6]:
plt.plot(recreated_signal2)
plt.title('Recreated Signal in Time Domain')
plt.show()

c:\Users\UserALai\miniconda3\lib\site-packages\matplotlib\cbook\__init__.py:1298: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


### Zeroing out parts of Cepstru WITHOUT windowing

In [9]:
csig = np.log(np.abs(np.fft.fft(S)))
icsig = np.real(np.fft.ifft(csig))
#icsig[int(len(icsig)/3):]=0
rsig = np.real(np.fft.ifft(np.exp(np.fft.fft(icsig)))) # recreated??
spectrogram_maker(rsig)
plt.show()